# Query Optimization

In [2]:
import psycopg2

In [10]:
conn = psycopg2.connect(
    host="traffic.cdq38kterv50.us-east-1.rds.amazonaws.com",
    database="traffic_crashes_v5",
    user="group4", 
    password="GroupNumberFour"
)

## Before Optimization

### Query 1

In [4]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
      WITH crash_city AS (
        SELECT DISTINCT
            c.crash_record_id,
            p.city_id
        FROM crash c
        JOIN person p ON c.crash_record_id = p.crash_record_id
        WHERE p.city_id IS NOT NULL 
      ),
      city_stats AS (
        SELECT
            ci.name AS city_name,
            COUNT(*) AS total_crashes,
            SUM(c.injuries_fatal) AS total_fatalities
        FROM crash_city cc
        JOIN crash c ON cc.crash_record_id = c.crash_record_id
        JOIN city ci ON cc.city_id = ci.id
        GROUP BY ci.name
        HAVING SUM(c.injuries_fatal) > 0
      )
      SELECT
        city_name,
        total_crashes,
        total_fatalities,
        ROUND(
            CASE WHEN total_crashes > 0
                THEN total_fatalities::NUMERIC / total_crashes
                ELSE 0 END,
            3
        ) AS severity_index
      FROM city_stats
      ORDER BY severity_index DESC
      LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('BALLSTON LANE', 2, 1, Decimal('0.500')),
 ('THE VILLAGES', 3, 1, Decimal('0.333')),
 ('MINERAL WELLS', 3, 1, Decimal('0.333')),
 ('MOUNTAIN VIEW', 15, 1, Decimal('0.067')),
 ('SHORELINE', 17, 1, Decimal('0.059'))]

CPU times: user 8.76 ms, sys: 2.67 ms, total: 11.4 ms
Wall time: 19.4 s


### Query 2

In [43]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
      WITH fatal_crashes AS (
          SELECT DISTINCT crash_record_id
          FROM crash
          WHERE injuries_fatal > 0
      )
      SELECT vt.name AS vehicle_type,
          COUNT(*) AS fatal_crash_count,
          ROUND(AVG(p.age), 1) AS avg_driver_age
      FROM fatal_crashes fc
      JOIN vehicle v ON fc.crash_record_id = v.crash_record_id
      JOIN vehicle_type vt ON v.vehicle_type_id = vt.id
      JOIN person p ON v.vehicle_id = p.vehicle_id
      JOIN person_type pt ON p.person_type_id = pt.id
      WHERE pt.name = 'DRIVER'
      GROUP BY vt.name
      ORDER BY fatal_crash_count DESC
""")
results = cursor.fetchall()
display(results)

[('PASSENGER', 426, Decimal('38.8')),
 ('SPORT UTILITY VEHICLE (SUV)', 81, Decimal('39.3')),
 ('VAN/MINI-VAN', 29, Decimal('38.9')),
 ('MOTORCYCLE (OVER 150CC)', 24, Decimal('49.5')),
 ('PICKUP', 23, Decimal('35.1')),
 ('UNKNOWN/NA', 21, Decimal('40.3')),
 ('TRUCK - SINGLE UNIT', 14, Decimal('43.2')),
 ('BUS OVER 15 PASS.', 9, Decimal('43.3')),
 ('TRACTOR W/ SEMI-TRAILER', 8, Decimal('36.0')),
 ('OTHER', 7, Decimal('41.1')),
 ('TRACTOR W/O SEMI-TRAILER', 2, Decimal('38.5')),
 ('SINGLE UNIT TRUCK WITH TRAILER', 2, Decimal('58.5')),
 ('BUS UP TO 15 PASS.', 1, Decimal('49.0'))]

CPU times: user 5.17 ms, sys: 2.62 ms, total: 7.79 ms
Wall time: 9.94 s


### Query 3

In [56]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT se.name AS safety_equipment,
          COUNT(*) AS total_drivers,
          SUM(CASE WHEN ic.name = 'FATAL' THEN 1 ELSE 0 END) AS fatal_drivers
      FROM person p
      JOIN person_type pt ON p.person_type_id = pt.id
      JOIN crash c ON p.crash_record_id = c.crash_record_id
      JOIN safety_equipment se ON p.safety_equipment_id = se.id
      JOIN injury_classification ic ON p.injury_classification_id = ic.id
      WHERE pt.name = 'DRIVER'
      GROUP BY se.name;
""")
results = cursor.fetchall()
display(results)

[('DOT COMPLIANT MOTORCYCLE HELMET', 860, 10),
 ('HELMET NOT USED', 1074, 18),
 ('HELMET USED', 284, 3),
 ('NONE PRESENT', 19754, 19),
 ('NOT DOT COMPLIANT MOTORCYCLE HELMET', 127, 3),
 ('SAFETY BELT NOT USED', 3400, 20),
 ('SAFETY BELT USED', 449889, 30),
 ('SHOULD/LAP BELT USED IMPROPERLY', 122, 0),
 ('STRETCHER', 2, 0),
 ('USAGE UNKNOWN', 278721, 144),
 ('WHEELCHAIR', 2, 0)]

CPU times: user 4.12 ms, sys: 2.39 ms, total: 6.52 ms
Wall time: 11.2 s


### Query 4

In [63]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
SELECT td.name AS travel_direction,
          COUNT(*) AS vehicle_count,
          SUM(c.injuries_fatal) AS total_fatalities
      FROM public.vehicle v
      JOIN public.travel_direction td ON v.travel_direction_id = td.id
      JOIN public.crash c ON v.crash_record_id = c.crash_record_id
      GROUP BY td.name
      ORDER BY vehicle_count DESC;
""")
results = cursor.fetchall()
display(results)

[('N', 351027, 334),
 ('S', 343282, 331),
 ('W', 312173, 247),
 ('E', 306640, 248),
 ('UNKNOWN', 65125, 79),
 ('SE', 19574, 12),
 ('NW', 17745, 13),
 ('SW', 14706, 5),
 ('NE', 14323, 14)]

CPU times: user 3.39 ms, sys: 2.04 ms, total: 5.43 ms
Wall time: 12.5 s


### Query 5

In [76]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT vt.name AS v,
          COUNT(*) AS vehicle_count,
          SUM(c.injuries_fatal) AS total_fatalities,
          ROUND(100.0 * SUM(c.injuries_fatal) / COUNT(*), 2) AS fatal_rate_pct
      FROM public.vehicle v
      JOIN public.vehicle_type vt ON v.vehicle_type_id = vt.id
      JOIN public.crash c ON v.crash_record_id = c.crash_record_id
      GROUP BY vt.name
      ORDER BY fatal_rate_pct DESC;
""")
results = cursor.fetchall()
display(results)

[('MOTORCYCLE (OVER 150CC)', 3279, 49, Decimal('1.49')),
 ('MOPED OR MOTORIZED BICYCLE', 309, 1, Decimal('0.32')),
 ('TRACTOR W/O SEMI-TRAILER', 1754, 5, Decimal('0.29')),
 ('PICKUP', 48534, 51, Decimal('0.11')),
 ('TRACTOR W/ SEMI-TRAILER', 13006, 14, Decimal('0.11')),
 ('SINGLE UNIT TRUCK WITH TRAILER', 1954, 2, Decimal('0.10')),
 ('UNKNOWN/NA', 38557, 34, Decimal('0.09')),
 ('PASSENGER', 982599, 856, Decimal('0.09')),
 ('VAN/MINI-VAN', 70817, 60, Decimal('0.08')),
 ('BUS OVER 15 PASS.', 16560, 14, Decimal('0.08')),
 ('TRUCK - SINGLE UNIT', 26270, 20, Decimal('0.08')),
 ('OTHER', 15541, 11, Decimal('0.07')),
 ('SPORT UTILITY VEHICLE (SUV)', 218273, 163, Decimal('0.07')),
 ('OTHER VEHICLE WITH TRAILER', 1907, 1, Decimal('0.05')),
 ('BUS UP TO 15 PASS.', 4657, 2, Decimal('0.04')),
 ('SNOWMOBILE', 1, 0, Decimal('0.00')),
 ('AUTOCYCLE', 196, 0, Decimal('0.00')),
 ('3-WHEELED MOTORCYCLE (2 REAR WHEELS)', 49, 0, Decimal('0.00')),
 ('ALL-TERRAIN VEHICLE (ATV)', 72, 0, Decimal('0.00')),
 ('R

CPU times: user 6.34 ms, sys: 2.37 ms, total: 8.71 ms
Wall time: 12.5 s


### Query 6

In [90]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT CASE
              WHEN c.crash_year - v.vehicle_year < 5 THEN 'Under 5'
              WHEN c.crash_year - v.vehicle_year BETWEEN 5 AND 10 THEN '5-10'
              WHEN c.crash_year - v.vehicle_year BETWEEN 10 AND 25 THEN '10-25'
              ELSE '25+' END AS vehicle_age,
          COUNT(*) AS vehicle_count,
          SUM(c.injuries_fatal) AS total_fatalities,
          ROUND(100.0 * SUM(c.injuries_fatal) / COUNT(*), 2) AS fatal_rate_pct
      FROM public.vehicle v
      JOIN public.vehicle_type vt ON v.vehicle_type_id = vt.id
      JOIN public.crash c ON v.crash_record_id = c.crash_record_id
      GROUP BY vehicle_age
      ORDER BY total_fatalities DESC;
""")
results = cursor.fetchall()
display(results)

[('10-25', 495834, 575, Decimal('0.12')),
 ('5-10', 477711, 365, Decimal('0.08')),
 ('Under 5', 460798, 322, Decimal('0.07')),
 ('25+', 10252, 21, Decimal('0.20'))]

CPU times: user 3.83 ms, sys: 2.88 ms, total: 6.71 ms
Wall time: 12.6 s


### Query 7

In [43]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT ct.name AS crash_type, COUNT(*) AS total_crashes, 
        SUM(c.injuries_fatal) AS total_fatalities,
            ROUND(100.0 * SUM(c.injuries_fatal) / COUNT(*), 2) AS fatality_rate_pct
        FROM public.crash c
        JOIN public.first_crash_type ct ON c.first_crash_type_id = ct.id
        WHERE c.crash_year = '2017'
        GROUP BY ct.name
        ORDER BY fatality_rate_pct DESC
        LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('PEDESTRIAN', 1776, 16, Decimal('0.90')),
 ('FIXED OBJECT', 3183, 27, Decimal('0.85')),
 ('HEAD ON', 748, 2, Decimal('0.27')),
 ('PEDALCYCLIST', 1255, 3, Decimal('0.24')),
 ('ANGLE', 8647, 6, Decimal('0.07'))]

CPU times: user 4.51 ms, sys: 6.17 ms, total: 10.7 ms
Wall time: 9.55 s


### Query 8

In [19]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT wc.name AS weather_condition, COUNT(*) AS total_crashes, SUM(c.injuries_fatal) AS fatal_crashes
      FROM public.crash c
      JOIN public.weather_condition wc ON c.weather_condition_id = wc.id
      GROUP BY wc.name
      ORDER BY total_crashes DESC
      LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('CLEAR', 702576, 682),
 ('RAIN', 77071, 93),
 ('UNKNOWN', 52521, 14),
 ('SNOW', 29495, 23),
 ('CLOUDY/OVERCAST', 26281, 18)]

CPU times: user 3.49 ms, sys: 2.77 ms, total: 6.26 ms
Wall time: 4.33 s


### Query 9

In [20]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
      SELECT pcc.name AS primary_cause, COUNT(*) AS fatal_crashes
      FROM public.crash c
      JOIN public.prim_contributory_cause pcc ON 
        c.prim_contributory_cause_id = pcc.id
      WHERE c.crash_year = '2015' AND c.injuries_fatal > 0 
      GROUP BY pcc.name
      ORDER BY fatal_crashes DESC
      LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('FAILING TO YIELD RIGHT-OF-WAY', 1),
 ('IMPROPER TURNING/NO SIGNAL', 1),
 ('PHYSICAL CONDITION OF DRIVER', 1)]

CPU times: user 3.09 ms, sys: 1.89 ms, total: 4.98 ms
Wall time: 4.06 s


### Query 10

In [21]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
      SELECT wc.name AS weather_condition, COUNT(*) AS total_crashes, SUM(c.injuries_fatal) AS fatal_crashes
      FROM public.crash c
      JOIN public.weather_condition wc ON c.weather_condition_id = wc.id
      WHERE c.crash_year = '2025'
      GROUP BY wc.name
      ORDER BY total_crashes DESC
      LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('CLEAR', 18857, 12),
 ('UNKNOWN', 2317, 1),
 ('RAIN', 1696, 1),
 ('SNOW', 1625, 1),
 ('CLOUDY/OVERCAST', 849, 0)]

CPU times: user 3.26 ms, sys: 2.39 ms, total: 5.65 ms
Wall time: 4.04 s


## Index Creation

In [ ]:
cursor = conn.cursor()

cursor.execute('''
CREATE INDEX idx_person_crash_city ON person(crash_record_id, city_id) WHERE city_id IS NOT NULL;
CREATE INDEX idx_city_name ON city(name);
CREATE INDEX idx_crash_injuries ON crash(crash_record_id, injuries_fatal);
CREATE INDEX idx_vehicle_crash_type_id ON vehicle(crash_record_id, vehicle_type_id, vehicle_id);
CREATE INDEX idx_person_vehicle_driver ON person(vehicle_id, person_type_id, age);
CREATE INDEX idx_person_type_driver ON person_type(id) WHERE name = 'DRIVER';
CREATE INDEX idx_vehicle_type_name ON vehicle_type(id, name);
CREATE INDEX idx_person_driver_safety ON person(person_type_id, safety_equipment_id, injury_classification_id, crash_record_id);
CREATE INDEX idx_crash_year ON crash(crash_year);
CREATE INDEX idx_vehicle_travel_crash ON vehicle(travel_direction_id, crash_record_id);
CREATE INDEX idx_vehicle_covering ON vehicle(travel_direction_id, crash_record_id, vehicle_id);
CREATE INDEX idx_travel_direction_covering ON travel_direction(id, name);
CREATE INDEX idx_vehicle_type_crash ON vehicle(vehicle_type_id, crash_record_id);
CREATE INDEX idx_vehicle_type ON vehicle(vehicle_type_id);
CREATE INDEX idx_vehicle_age_crash ON vehicle(crash_record_id, vehicle_year, vehicle_type_id);
CREATE INDEX idx_crash_year_fatal ON crash(crash_record_id, crash_year, injuries_fatal);
CREATE INDEX idx_crash_type_fatalities ON crash(first_crash_type_id, injuries_fatal);
CREATE INDEX idx_first_crash_type_id_name ON first_crash_type(id, name);
CREATE INDEX idx_crash_weather_fatalities ON crash(weather_condition_id, injuries_fatal);
CREATE INDEX idx_weather_condition_id_name ON weather_condition(id, name);
CREATE INDEX idx_crash_filters ON crash(posted_speed_limit, injuries_total, crash_hour, crash_record_id);
CREATE INDEX idx_weather_condition_name ON weather_condition(name);
CREATE INDEX idx_lighting_condition_name ON lighting_condition(name);
CREATE INDEX idx_first_crash_type_name ON first_crash_type(name);
CREATE INDEX idx_trafficway_type_name ON trafficway_type(name);
CREATE INDEX idx_roadway_surface_condition_name ON roadway_surface_condition(name);
CREATE INDEX idx_damage_name ON damage(name);
CREATE INDEX idx_safety_equipment_name ON safety_equipment(name);
CREATE INDEX idx_crash_search_filters ON crash(posted_speed_limit, injuries_total, crash_hour);
CREATE INDEX idx_crash_record_id_text ON crash(crash_record_id text_pattern_ops);
CREATE INDEX idx_crash_weather_condition_id ON crash(weather_condition_id);
CREATE INDEX idx_crash_lighting_condition_id ON crash(lighting_condition_id);
CREATE INDEX idx_crash_crash_type_id ON crash(crash_type_id);
CREATE INDEX idx_crash_trafficway_type_id ON crash(trafficway_type_id);
CREATE INDEX idx_crash_roadway_surface_cond_id ON crash(roadway_surface_cond_id);
CREATE INDEX idx_crash_damage_id ON crash(damage_id);

''')

conn.commit()

## Materialized Views

In [46]:
cursor = conn.cursor()

cursor.execute('''
        CREATE MATERIALIZED VIEW mv_city_crash_severity AS
        SELECT c.crash_year,
            ci.name AS city_name,
            COUNT(DISTINCT c.crash_record_id) AS total_crashes,
            SUM(c.injuries_fatal) AS total_fatalities,
            ROUND(SUM(c.injuries_fatal)::NUMERIC / COUNT(DISTINCT c.crash_record_id), 3) AS severity_index
        FROM crash c
        JOIN person p ON c.crash_record_id = p.crash_record_id
        JOIN city ci ON p.city_id = ci.id
        WHERE p.city_id IS NOT NULL
        GROUP BY c.crash_year, ci.name
        HAVING SUM(c.injuries_fatal) > 0;
''')

conn.commit()

In [67]:
cursor = conn.cursor()

cursor.execute('''
    CREATE MATERIALIZED VIEW mv_vehicle_type_fatal_driver_stats AS
    SELECT vt.name AS vehicle_type,
        COUNT(*) AS fatal_crash_count,
        ROUND(AVG(p.age), 1) AS avg_driver_age
    FROM crash c
    JOIN vehicle v ON c.crash_record_id = v.crash_record_id
    JOIN vehicle_type vt ON v.vehicle_type_id = vt.id
    JOIN person p ON v.vehicle_id = p.vehicle_id
    JOIN person_type pt ON p.person_type_id = pt.id
    WHERE c.injuries_fatal > 0 
    AND pt.name = 'DRIVER'
    GROUP BY vt.name;
''')

conn.commit()

In [58]:
cursor = conn.cursor()

cursor.execute('''
    CREATE MATERIALIZED VIEW mv_travel_direction_stats AS
    SELECT c.crash_year,
        td.name AS travel_direction,
        COUNT(*) AS vehicle_count,
        SUM(c.injuries_fatal) AS total_fatalities
    FROM vehicle v
    JOIN travel_direction td ON v.travel_direction_id = td.id
    JOIN crash c ON v.crash_record_id = c.crash_record_id
    GROUP BY c.crash_year, td.name;
''')

conn.commit()

In [27]:
cursor = conn.cursor()

cursor.execute('''
    CREATE MATERIALIZED VIEW mv_vehicle_fatality_stats AS
        SELECT vt.name AS v,
            COUNT(*) AS vehicle_count,
            SUM(c.injuries_fatal) AS total_fatalities,
            ROUND(100.0 * SUM(c.injuries_fatal) / COUNT(*), 2) AS fatal_rate_pct
        FROM vehicle v
        JOIN vehicle_type vt ON v.vehicle_type_id = vt.id
        JOIN crash c ON v.crash_record_id = c.crash_record_id
        GROUP BY vt.name;
''')

conn.commit()

In [28]:
cursor = conn.cursor()

cursor.execute('''
    CREATE MATERIALIZED VIEW mv_vehicle_age_stats AS
        SELECT vt.name AS vehicle_type,
            CASE
                WHEN c.crash_year - v.vehicle_year < 5 THEN 'Under 5'
                WHEN c.crash_year - v.vehicle_year BETWEEN 5 AND 10 THEN '5-10'
                WHEN c.crash_year - v.vehicle_year BETWEEN 10 AND 25 THEN '10-25'
                ELSE '25+' END AS vehicle_age,
            COUNT(*) AS vehicle_count,
            SUM(c.injuries_fatal) AS total_fatalities,
            ROUND(100.0 * SUM(c.injuries_fatal) / COUNT(*), 2) AS fatal_rate_pct
        FROM vehicle v
        JOIN vehicle_type vt ON v.vehicle_type_id = vt.id
        JOIN crash c ON v.crash_record_id = c.crash_record_id
        GROUP BY vt.name, vehicle_age;
''')

conn.commit()

In [29]:
cursor = conn.cursor()

cursor.execute('''
    CREATE MATERIALIZED VIEW mv_safety_equipment_driver_stats AS
        SELECT se.name AS safety_equipment,
            COUNT(*) AS total_drivers,
            SUM(CASE WHEN ic.name = 'FATAL' THEN 1 ELSE 0 END) AS fatal_drivers
        FROM person p
        JOIN safety_equipment se ON p.safety_equipment_id = se.id
        JOIN injury_classification ic ON p.injury_classification_id = ic.id
        WHERE p.person_type_id = (SELECT id FROM person_type WHERE name = 'DRIVER')
        GROUP BY se.name;
''')

conn.commit()

In [30]:
cursor = conn.cursor()

cursor.execute('''
    CREATE MATERIALIZED VIEW mv_primary_cause_fatal_stats AS
    SELECT c.crash_year,
        pcc.name AS primary_cause, 
        COUNT(*) AS fatal_crashes
    FROM crash c
    JOIN prim_contributory_cause pcc ON c.prim_contributory_cause_id = pcc.id
    WHERE c.injuries_fatal > 0
    GROUP BY c.crash_year, pcc.name;
''')

conn.commit()

In [31]:
cursor = conn.cursor()

cursor.execute('''
    CREATE MATERIALIZED VIEW mv_weather_condition_stats AS
        SELECT c.crash_year,
            wc.name AS weather_condition, 
            COUNT(*) AS total_crashes, 
            SUM(c.injuries_fatal) AS fatal_crashes
        FROM crash c
        JOIN weather_condition wc ON c.weather_condition_id = wc.id
        GROUP BY c.crash_year, wc.name;
''')

conn.commit()

In [44]:
cursor = conn.cursor()

cursor.execute('''
    CREATE MATERIALIZED VIEW mv_crash_type_fatality_stats AS
        SELECT c.crash_year,
            ct.name AS crash_type, 
            COUNT(*) AS total_crashes, 
            SUM(c.injuries_fatal) AS total_fatalities,
            ROUND(100.0 * SUM(c.injuries_fatal) / COUNT(*), 2) AS fatality_rate_pct
        FROM crash c
        JOIN first_crash_type ct ON c.first_crash_type_id = ct.id
        GROUP BY c.crash_year, ct.name;
''')

conn.commit()

## Rollback Optimizations

In [ ]:
# cursor = conn.cursor()

# cursor.execute('''
#     DROP INDEX IF EXISTS idx_person_crash_city;
#     DROP INDEX IF EXISTS idx_city_name;
#     DROP INDEX IF EXISTS idx_crash_optimized; -- remove
#     DROP INDEX IF EXISTS idx_crash_injuries;
#     DROP INDEX IF EXISTS idx_vehicle_crash_type_id;
#     DROP INDEX IF EXISTS idx_person_vehicle_driver;
#     DROP INDEX IF EXISTS idx_person_type_driver;
#     DROP INDEX IF EXISTS idx_vehicle_type_name;
#     DROP INDEX IF EXISTS idx_person_driver_safety;
#     DROP INDEX IF EXISTS idx_crash_year;
#     DROP INDEX IF EXISTS idx_vehicle_travel_crash;
#     DROP INDEX IF EXISTS idx_vehicle_covering;
#     DROP INDEX IF EXISTS idx_travel_direction_covering;
#     DROP INDEX IF EXISTS idx_vehicle_type_crash;
#     DROP INDEX IF EXISTS idx_vehicle_type;
#     DROP INDEX IF EXISTS idx_vehicle_age_crash;
#     DROP INDEX IF EXISTS idx_crash_year_fatal;
#     DROP INDEX IF EXISTS idx_crash_type_fatalities;
#     DROP INDEX IF EXISTS idx_first_crash_type_id_name;
#     DROP INDEX IF EXISTS idx_crash_weather_fatalities;
#     DROP INDEX IF EXISTS idx_weather_condition_id_name;
#     DROP INDEX IF EXISTS idx_crash_filters;
#     DROP INDEX IF EXISTS idx_weather_condition_name;
#     DROP INDEX IF EXISTS idx_lighting_condition_name;
#     DROP INDEX IF EXISTS idx_first_crash_type_name;
#     DROP INDEX IF EXISTS idx_trafficway_type_name;
#     DROP INDEX IF EXISTS idx_roadway_surface_condition_name;
#     DROP INDEX IF EXISTS idx_damage_name;
#     DROP INDEX IF EXISTS idx_crash_search_filters;
#     DROP INDEX IF EXISTS idx_crash_record_id_text;
#     DROP INDEX IF EXISTS idx_crash_weather_condition_id;
#     DROP INDEX IF EXISTS idx_crash_lighting_condition_id;
#     DROP INDEX IF EXISTS idx_crash_crash_type_id;
#     DROP INDEX IF EXISTS idx_crash_trafficway_type_id;
#     DROP INDEX IF EXISTS idx_crash_roadway_surface_cond_id;
#     DROP INDEX IF EXISTS idx_crash_damage_id;
#     DROP MATERIALIZED VIEW IF EXISTS mv_city_crash_severity;
#     DROP MATERIALIZED VIEW IF EXISTS mv_vehicle_type_fatal_driver_stats;
#     DROP MATERIALIZED VIEW IF EXISTS mv_travel_direction_stats;
#     DROP MATERIALIZED VIEW IF EXISTS mv_vehicle_fatality_stats;
#     DROP MATERIALIZED VIEW IF EXISTS mv_vehicle_age_stats;
#     DROP MATERIALIZED VIEW IF EXISTS mv_safety_equipment_driver_stats;
#     DROP MATERIALIZED VIEW IF EXISTS mv_primary_cause_fatal_stats;
#     DROP MATERIALIZED VIEW IF EXISTS mv_weather_condition_stats;
#     DROP MATERIALIZED VIEW IF EXISTS mv_crash_type_fatality_stats;
# ''')

# conn.commit()

## After Optimization

### Query 1

In [48]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT city_name, total_crashes, total_fatalities, severity_index
     FROM mv_city_crash_severity 
     WHERE crash_year = '2017'
     ORDER BY severity_index DESC
     LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('ROBBINS', 27, 1, Decimal('0.037')),
 ('VERNON HILLS', 52, 1, Decimal('0.019')),
 ('RICHTON PARK', 100, 1, Decimal('0.010')),
 ('JUSTICE', 109, 1, Decimal('0.009')),
 ('HOMEWOOD', 149, 1, Decimal('0.007'))]

CPU times: user 3.25 ms, sys: 2.69 ms, total: 5.94 ms
Wall time: 169 ms


### Query 2

In [68]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT vehicle_type, fatal_crash_count, avg_driver_age
     FROM mv_vehicle_type_fatal_driver_stats 
     ORDER BY fatal_crash_count DESC;
""")
results = cursor.fetchall()
display(results)

[('PASSENGER', 426, Decimal('38.8')),
 ('SPORT UTILITY VEHICLE (SUV)', 81, Decimal('39.3')),
 ('VAN/MINI-VAN', 29, Decimal('38.9')),
 ('MOTORCYCLE (OVER 150CC)', 24, Decimal('49.5')),
 ('PICKUP', 23, Decimal('35.1')),
 ('UNKNOWN/NA', 21, Decimal('40.3')),
 ('TRUCK - SINGLE UNIT', 14, Decimal('43.2')),
 ('BUS OVER 15 PASS.', 9, Decimal('43.3')),
 ('TRACTOR W/ SEMI-TRAILER', 8, Decimal('36.0')),
 ('OTHER', 7, Decimal('41.1')),
 ('TRACTOR W/O SEMI-TRAILER', 2, Decimal('38.5')),
 ('SINGLE UNIT TRUCK WITH TRAILER', 2, Decimal('58.5')),
 ('BUS UP TO 15 PASS.', 1, Decimal('49.0'))]

CPU times: user 3.53 ms, sys: 2.2 ms, total: 5.73 ms
Wall time: 396 ms


### Query 3

In [7]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    WITH drivers AS (
     SELECT p.*, se.name AS safety_equipment, ic.name AS injury_level
     FROM person p
     JOIN person_type pt ON p.person_type_id = pt.id
     JOIN safety_equipment se ON p.safety_equipment_id = se.id
     JOIN injury_classification ic ON p.injury_classification_id = ic.id
     WHERE pt.name = 'DRIVER' AND se.name = 'SAFETY BELT NOT USED'
 )SELECT
          CASE
              WHEN age < 25 THEN 'Under 25'
              WHEN age BETWEEN 25 AND 64 THEN '25-64'
              ELSE '65+' END AS age_group,
          COUNT(*) AS driver_count,
          SUM(CASE WHEN injury_level = 'FATAL' THEN 1 ELSE 0 END) AS fatalities
      FROM drivers
      GROUP BY age_group
      ORDER BY fatalities DESC;
""")
results = cursor.fetchall()
display(results)

[('25-64', 2491, 12), ('Under 25', 676, 5), ('65+', 233, 3)]

CPU times: user 3 ms, sys: 2.14 ms, total: 5.14 ms
Wall time: 605 ms


In [154]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT * FROM mv_safety_equipment_driver_stats
    WHERE safety_equipment = 'DOT COMPLIANT MOTORCYCLE HELMET'
""")
results = cursor.fetchall()
display(results)

[('DOT COMPLIANT MOTORCYCLE HELMET', 860, 10)]

CPU times: user 1.83 ms, sys: 1.29 ms, total: 3.11 ms
Wall time: 197 ms


### Query 4

In [59]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT travel_direction, vehicle_count, total_fatalities
     FROM mv_travel_direction_stats 
     WHERE crash_year = '2025'
     ORDER BY vehicle_count DESC
""")
results = cursor.fetchall()
display(results)

[('N', 10355, 10),
 ('S', 10031, 1),
 ('W', 9270, 10),
 ('E', 8975, 2),
 ('UNKNOWN', 2429, 2),
 ('SE', 628, 0),
 ('NW', 544, 0),
 ('SW', 513, 0),
 ('NE', 451, 0)]

CPU times: user 2.72 ms, sys: 1.26 ms, total: 3.98 ms
Wall time: 345 ms


### Query 5

In [53]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT * FROM mv_vehicle_fatality_stats 
    WHERE v = 'PASSENGER'
    ORDER BY fatal_rate_pct DESC;
""")
results = cursor.fetchall()
display(results)

[('PASSENGER', 982599, 856, Decimal('0.09'))]

CPU times: user 3.04 ms, sys: 2.29 ms, total: 5.33 ms
Wall time: 345 ms


### Query 6

In [96]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT * FROM mv_vehicle_age_stats ORDER BY total_fatalities DESC;
""")
results = cursor.fetchall()
display(results)

[('10-25', 495834, 575, Decimal('0.12')),
 ('5-10', 477711, 365, Decimal('0.08')),
 ('Under 5', 460798, 322, Decimal('0.07')),
 ('25+', 10252, 21, Decimal('0.20'))]

CPU times: user 2.16 ms, sys: 2 ms, total: 4.16 ms
Wall time: 335 ms


### Query 7

In [45]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT crash_type, total_crashes, total_fatalities, fatality_rate_pct
     FROM mv_crash_type_fatality_stats 
     WHERE crash_year = '2017'
     ORDER BY fatality_rate_pct DESC
     LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('PEDESTRIAN', 1776, 16, Decimal('0.90')),
 ('FIXED OBJECT', 3183, 27, Decimal('0.85')),
 ('HEAD ON', 748, 2, Decimal('0.27')),
 ('PEDALCYCLIST', 1255, 3, Decimal('0.24')),
 ('ANGLE', 8647, 6, Decimal('0.07'))]

CPU times: user 1.81 ms, sys: 2.11 ms, total: 3.92 ms
Wall time: 346 ms


### Query 8

In [122]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT wc.name AS weather_condition, COUNT(*) AS total_crashes, SUM(c.injuries_fatal) AS fatal_crashes
      FROM public.crash c
      JOIN public.weather_condition wc ON c.weather_condition_id = wc.id
      GROUP BY wc.name
      ORDER BY total_crashes DESC
      LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('CLEAR', 702576, 682),
 ('RAIN', 77071, 93),
 ('UNKNOWN', 52521, 14),
 ('SNOW', 29495, 23),
 ('CLOUDY/OVERCAST', 26281, 18)]

CPU times: user 2.03 ms, sys: 1.92 ms, total: 3.95 ms
Wall time: 537 ms


### Query 9

In [32]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
    SELECT primary_cause, fatal_crashes
     FROM mv_primary_cause_fatal_stats 
     WHERE crash_year = '2025'
     ORDER BY fatal_crashes DESC
     LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('UNABLE TO DETERMINE', 9),
 ('FAILING TO YIELD RIGHT-OF-WAY', 2),
 ('IMPROPER OVERTAKING/PASSING', 2),
 ('DISREGARDING TRAFFIC SIGNALS', 1),
 ('OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
  1)]

CPU times: user 2.44 ms, sys: 2.76 ms, total: 5.2 ms
Wall time: 341 ms


### Query 10

In [33]:
%%time
cursor = conn.cursor()
cursor.execute(f"""
      SELECT weather_condition, total_crashes, fatal_crashes
      FROM mv_weather_condition_stats 
      WHERE crash_year = '2025'
      ORDER BY total_crashes DESC
      LIMIT 5;
""")
results = cursor.fetchall()
display(results)

[('CLEAR', 18857, 12),
 ('UNKNOWN', 2317, 1),
 ('RAIN', 1696, 1),
 ('SNOW', 1625, 1),
 ('CLOUDY/OVERCAST', 849, 0)]

CPU times: user 1.73 ms, sys: 1.35 ms, total: 3.09 ms
Wall time: 221 ms
